In [1]:
!pip install datasets sentence-transformers

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [2]:
!tar -xvzf colPT.tgz

tar (child): colPT.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


### Load case reports

In [19]:
import os
import re
dir = "./colPT"
data = []
#[{
# "cdu":["743.3","..."]
# "title":"titulo..."
# "abstract":"........."
#}...
#]
for filename in list(os.listdir(dir)):
    with open(os.path.join(dir, filename), encoding="UTF8") as f:
        text = f.read()
        file_info = {}
        for key, value in re.findall(r"#(\w+) --\s*(.*)",text):
            if key == "sub":
                file_info["sub"] = re.split(r"\s*\|\s*", value.strip("."))
            else:
                file_info[key] = value
        file_info["text"] = re.sub(r"#.*\n","",text)
        data.append(file_info)

         

In [2]:
data[:10]

[{'aut': 'Carvalho, Júlia Elisabete Oliveira Macedo.',
  'tit': 'Decomposição fotocatalítica de corantes orgânicos utilizados na indústria têxtil.',
  'date': '2009-08-04.',
  'sub': ['628.543:677', '677:628.543'],
  'text': 'A procura de tecnologias efectivas tendo em vista o tratamento dos efluentes têxteis tem-se intensificado no sentido de resolver este problema ambiental. Neste contexto, os Processos Oxidativos Avançados (POAs) surgem como tecnologias inovadoras baseadas na formação de espécies reactivas, tais como os radicais hidroxilo (HO•). Estes radicais são extremamente reactivos, apresentam reduzida selectividade e são capazes de oxidar uma grande diversidade de poluentes orgânicos. Dos POAs, destaca-se a degradação fotocatalítica devido à elevada eficiência que tem vindo a demonstrar no que refere à degradação de compostos orgânicos. Este processo fotocatalítico tem início com a activação de um semicondutor através da absorção de radiação solar ou artificial, formando elect

In [ ]:
from itertools import combinations

pairs = []

### slice data to generate a smaller training set due to memory constraints
small_data = data[:1000] # there are smarter ways to do this e.g. stratified sampling
for data1, data2 in combinations(small_data,2):
    score = len(set(data1["sub"]) & set(data2["sub"]))
    pairs.append((data1["text"],data2["text"],score if score > 1 else 0))
#[(tex1,tex2,score)]

In [ ]:
len(pairs), len(small_data)

(5928846, 3444)

In [5]:
from collections import Counter
scores = Counter([score for _, _, score in pairs])
print(scores)

Counter({0: 5921439, 2: 6861, 3: 400, 4: 101, 5: 22, 6: 18, 7: 4, 9: 1})


### Balance similar and non-similar pairs

In [6]:
from sklearn.utils import resample

majority_class = [pair for pair in pairs if pair[2] == 0]
minority_class = [pair for pair in pairs if pair[2] != 0]

undersampled_majority_class = resample(majority_class,
                                       replace=False,     # Don't duplicate samples
                                       n_samples= len(minority_class),  # Match minority class size
                                       random_state=42)

balanced_pairs = undersampled_majority_class + minority_class

# count pairs by score
score_counter = Counter([score for _, _, score in balanced_pairs])
score_counter

Counter({0: 7407, 2: 6861, 3: 400, 4: 101, 6: 18, 5: 22, 7: 4, 9: 1})

### Similarity score based on the number of common keywords

In [7]:
def normalize_score(score):
    if score == 0:
        return 0
    if score == 2:
        return 0.5
    if score == 3:
        return 0.75
    if score == 4:
        return 0.85
    if score > 4:
        return 1
    
balanced_pairs_norm = [(t1,t2,normalize_score(score)) for t1,t2,score in balanced_pairs]

print(balanced_pairs_norm[13000])

('Esta dissertação pretende perceber como as crianças de 4 e 5 anos entendem as relações\nlógicas da divisão partitiva de quantidades discretas. Procurou-se saber como as crianças\npequenas compreendem a relação inversa entre divisor e quociente, quando o dividendo se\nmantém constante; e ainda, como entendem a divisão de quantidades discretas em partes\niguais.\nEste estudo incidiu num grupo de crianças de 4 e 5 anos de Jardins de Infância do\nconcelho de Esposende, Braga. As crianças realizaram tarefas de divisão partitiva de\nquantidades discretas envolvendo conjuntos de 12 e 24 unidades. Os dados foram recolhidos a\npartir de entrevistas individuais, estruturadas, tendo-se recorrido à gravação áudio e vídeo.\nUtilizou-se uma metodologia quantitativa na análise dos dados. Esta análise centrou-se nas\nestimativas das crianças para o quociente nas divisões, nos procedimentos por elas utilizados,\nnas suas justificações e nos desempenhos por elas apresentados.\nOs resultados indicam qu

### Generate test and train split

In [11]:
from sklearn.model_selection import train_test_split

scores = [score for _, _, score in balanced_pairs_norm]

train_data, test_data = train_test_split(
balanced_pairs_norm,
test_size=0.2,
random_state=42,
stratify=scores
)

score_train= Counter([score for _, _, score in train_data])
score_test= Counter([score for _, _, score in test_data])
print(score_train)
print(score_test)

Counter({0: 5925, 0.5: 5489, 0.75: 320, 0.85: 81, 1: 36})
Counter({0: 1482, 0.5: 1372, 0.75: 80, 0.85: 20, 1: 9})


In [12]:
from datasets import Dataset

def convert_to_dict_of_lists(data_tuples):

    result = {
        'text1': [],
        'text2': [],
        'score': [],
    }

    for abstract1, abstract2, score in data_tuples:
        result['text1'].append(abstract1)
        result['text2'].append(abstract2)
        result['score'].append(score)

    return result

train_dataset = Dataset.from_dict(convert_to_dict_of_lists(train_data))
test_dataset = Dataset.from_dict(convert_to_dict_of_lists(test_data))


# Check the datasets
print("Train Dataset:", train_dataset)
print("Validation Dataset:", test_dataset)


/home/lfc/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train Dataset: Dataset({
    features: ['text1', 'text2', 'score'],
    num_rows: 11851
})
Validation Dataset: Dataset({
    features: ['text1', 'text2', 'score'],
    num_rows: 2963
})


### Train the Bi-encoder model

In [14]:
from sentence_transformers import SentenceTransformer, losses


model = SentenceTransformer('neuralmind/bert-base-portuguese-cased')
loss= losses.CosineSimilarityLoss(model)

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.
/home/lfc/.local/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.similarity_functions import SimilarityFunction
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator



args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/my_model",
    report_to="none",
    # Optional training parameters:
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)


# Create the evaluator
dev_evaluator = EmbeddingSimilarityEvaluator(
    test_dataset['text1'],  # Assuming these are the sentence pairs for evaluation
    test_dataset['text2'],
    test_dataset['score'],  # Assuming this contains the similarity scores
    main_similarity=SimilarityFunction.COSINE,
)

# 6. Create the trainer & start training
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)



In [16]:
# check if um using gpu
import torch
torch.cuda.is_available()

True

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson Cosine,Spearman Cosine,Pearson Manhattan,Spearman Manhattan,Pearson Euclidean,Spearman Euclidean,Pearson Dot,Spearman Dot,Pearson Max,Spearman Max
1,No log,0.025142,0.883483,0.833967,0.846610,0.816250,0.848135,0.816603,0.854170,0.827014,0.883483,0.833967
2,0.025000,0.019887,0.908951,0.847177,0.875832,0.839710,0.876548,0.839385,0.896363,0.842105,0.908951,0.847177
3,0.025000,0.017539,0.919985,0.850829,0.879777,0.841347,0.880518,0.840852,0.909485,0.847726,0.919985,0.850829
4,0.005200,0.017185,0.922005,0.853252,0.889898,0.846918,0.890807,0.846534,0.911146,0.849017,0.922005,0.853252
5,0.001600,0.017048,0.922984,0.853104,0.890641,0.846424,0.891358,0.845941,0.912774,0.848968,0.922984,0.853104


TrainOutput(global_step=1505, training_loss=0.010598825289252094, metrics={'train_runtime': 9715.5613, 'train_samples_per_second': 2.476, 'train_steps_per_second': 0.155, 'total_flos': 0.0, 'train_loss': 0.010598825289252094, 'epoch': 5.0})

### Test bi-encoder model

In [ ]:


test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=test_dataset["text1"],
    sentences2=test_dataset["text2"],
    scores=test_dataset["score"],
    main_similarity=SimilarityFunction.COSINE,
)
test_evaluator(model)



{'sts-test_pearson_cosine': 0.8871676506901097,
 'sts-test_spearman_cosine': 0.8432246765443026}

In [ ]:


# 8. Save the trained model
model.save_pretrained("directory_path")

### Infernece

In [18]:
data = []

In [20]:
abstracts = [entry["text"] for entry in data]
titles = [entry["tit"] for entry in data]
len(abstracts), len(titles)

(3444, 3444)

In [ ]:
embeddings = model.encode(abstracts, convert_to_tensor=True)

In [ ]:
query_text = """Neste momento, a grande maioria dos arquivos portugueses com presença online utilizam
como plataforma de base o Digitarq (arquivos distritais dependentes da Direção Geral) ou o
Archeevo (todos os outros: municipais, ministeriais, empresariais, etc).
Esta dissertação, tem como objetivo implementar uma solução que permita interpretar
os catálogos dos arquivos e, às vezes, as transcrições dos documentos lá custodiados,
aplicando algoritmos de Machine Learning para a extração de entidades. As entidades
extraídas deverão ser guardadas numa ontologia que deve ser especificada (com um critério
importante: simplicidade).
Por fim, pretende-se desenvolver uma plataforma Web que seja responsável por servir o
projecto desenvolvido"""

In [ ]:
from sentence_transformers import util
import torch

query_embedding = model.encode(query_text, convert_to_tensor=True)

# Calculate the similarity between the query and the abstracts
cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)
retrieval_results = torch.topk(cosine_scores, k=15)

for score, idx in zip(retrieval_results.values[0], retrieval_results.indices[0]):

    print(f"Title: {titles[idx]}\nSimilarity Score: {score.item():.4f}\nSentence: {abstracts[idx]} \n")
    print("-" * 80)

Title: Benchmark de base de dados de suporte a serviços de informação.
Similarity Score: 0.7824
Sentence: ser feita tendo consciência das situações onde cada um é potencialmente mais indicado.
onde o modelo relacional poderá ser inferior, pelo que selecção do modelo a usar terá de

entanto, conclui-se que há situações eventualmente mais favoráveis para o uso de XML,

Os resultados obtidos apontam para um desempenho superior do modelo relacional. No

dar resposta às necessidades deste trabalho.

um sistema de testes, baseado em sistemas existentes mas construído de raiz com vista a

ao uso da outra num contexto dos serviços de informação. Isto é feito com o recurso a

objectivo de identificar situações onde o uso de uma poderá ter vantagens relativamente

uma análise comparativa de desempenho, sendo definido um benchmark, com o

relacional em oposição ao XML, são extremamente relevantes, neste trabalho é feita

Uma vez que estas duas abordagens para o armazenamento de informação, o mode